# Payoff Forward-start

Strike = m x S_start.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ipywidgets as widgets
import sys
from pathlib import Path
from IPython.display import display, Markdown

_base = Path.cwd().resolve()
for extra in (_base, _base.parent, _base.parent.parent, _base.parent.parent.parent):
    candidate = extra / "scripts"
    if candidate.exists() and str(candidate) not in sys.path:
        sys.path.insert(0, str(candidate))
        break
    candidate = extra / "notebooks" / "scripts"
    if candidate.exists() and str(candidate) not in sys.path:
        sys.path.insert(0, str(candidate))
        break
plt.style.use('seaborn-v0_8-darkgrid')

from pricing import fetch_spy_history, view_forward_start, payoff_forward_start

try:
    close_spy = fetch_spy_history()
except Exception:
    close_spy = None

if close_spy is not None and not getattr(close_spy, "empty", True):
    spot_ref = float(close_spy.iloc[-1])
else:
    spot_ref = 100.0
    close_spy = pd.Series([spot_ref], index=pd.Index([pd.Timestamp.today()]), name="Close")

spot_start = spot_ref
m_mult = 1.0


D:\PythonDProjects\PaperTradingApp\notebooks\scripts\pricing.py:47: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("SPY", period=period, interval=interval, progress=False)
C:\Users\nicol\AppData\Local\Temp\ipykernel_31136\2868333958.py:22: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  spot_ref = float(close_spy.iloc[-1])


## Payoff interactif
Spot de depart, multiplicateur m, sigma, r, T, call/put.


In [ ]:
sigma_slider = widgets.FloatSlider(value=0.2, min=0.01, max=1.0, step=0.01, description='Sigma')
r_slider = widgets.FloatSlider(value=0.02, min=-0.05, max=0.1, step=0.005, description='r')
T_slider = widgets.FloatSlider(value=1.0, min=0.05, max=2.0, step=0.05, description='T')

start_slider = widgets.FloatSlider(value=spot_ref, min=spot_ref*0.5, max=spot_ref*1.5, step=1.0, description='S_start')
m_slider = widgets.FloatSlider(value=1.0, min=0.5, max=1.5, step=0.05, description='m (mult strike)')
call_put = widgets.Dropdown(options=[('Call','call'),('Put','put')], value='call', description='Type')
output = widgets.Output()

def _update(change=None):
    with output:
        output.clear_output()
        s_start, m = start_slider.value, m_slider.value
        k_eff = m * s_start

        fig_ts, ax_ts = plt.subplots(figsize=(8, 3))
        ax_ts.plot(close_spy.index, close_spy.values, label='SPY close (1y)')
        ax_ts.axhline(s_start, color='gray', linestyle='--', label=f'S_start = {s_start:.2f}')
        ax_ts.axhline(k_eff, color='firebrick', linestyle=':', label=f'K = m*S_start = {k_eff:.2f}')
        ax_ts.set_ylabel('Prix')
        ax_ts.set_title('Clotures SPY (forward-start)')
        ax_ts.legend(loc='best')
        fig_ts.autofmt_xdate()
        plt.show()
        plt.close(fig_ts)

        view_dyn = view_forward_start(spot_ref, s_start, m=m, option_type=call_put.value, r=r_slider.value, q=0.0, sigma=sigma_slider.value, T=T_slider.value)
        premium = float(view_dyn.get('premium', 0.0))
        s_grid = view_dyn['s_grid']
        payoff_grid = view_dyn['payoff']
        pnl_grid = view_dyn['pnl']
        payoff_s0 = float(np.interp(spot_ref, s_grid, payoff_grid))
        pnl_s0 = payoff_s0 - premium

        fig, ax = plt.subplots(figsize=(7,4))
        ax.plot(s_grid, payoff_grid, label='Payoff')
        ax.plot(s_grid, pnl_grid, label='P&L net', color='darkorange')
        ax.axvline(spot_ref, color='gray', linestyle='--', label=f'S0 = {spot_ref:.2f}')
        ax.axhline(0, color='black', linewidth=0.8)
        ax.legend(loc='best')
        ax.set_xlabel('Spot')
        ax.set_ylabel('Payoff / P&L')
        ax.set_title('Forward-start (approx)')
        plt.show()
        plt.close(fig)
        display(Markdown(f"Prime ~ {premium:.4f}

- Payoff @ S0 = {payoff_s0:.4f}
- P&L net = {pnl_s0:.4f}"))

for sl in (start_slider, m_slider, sigma_slider, r_slider, T_slider, call_put):
    sl.observe(_update, names='value')
_update()
display(widgets.VBox([start_slider, m_slider, sigma_slider, r_slider, T_slider, call_put, output]))
